In [1]:
!pip install googletrans
!pip install pydub
!pip3 install SpeechRecognition
!pip install librosa

     |████████████████████████████████| 61kB 4.3MB/s 
     |████████████████████████████████| 51kB 6.0MB/s 
     |████████████████████████████████| 962kB 29.3MB/s 
     |████████████████████████████████| 61kB 8.1MB/s 
     |████████████████████████████████| 71kB 8.0MB/s 
     |████████████████████████████████| 102kB 11.4MB/s 
  Created wheel for googletrans: filename=googletrans-3.0.0-cp36-none-any.whl size=15736 sha256=750b4c3849ed53d923ea33b42787fe3b3274d97a47db0f5b5ccbcc9f888ddab3
  Stored in directory: /root/.cache/pip/wheels/28/1a/a7/eaf4d7a3417a0c65796c547cff4deb6d79c7d14c2abd29273e
  Created wheel for contextvars: filename=contextvars-2.4-cp36-none-any.whl size=7666 sha256=96c19775b86a6c65b6101cc7e0af0e0254f04f94afd5efc47c363affd0f8a84b
  Stored in directory: /root/.cache/pip/wheels/a5/7d/68/1ebae2668bda2228686e3c1cf16f2c2384cea6e9334ad5f6de
Successfully built googletrans contextvars
     |████████████████████████████████| 32.8MB 128kB/s 


In [2]:
!pip install flask

In [3]:
from google.colab import files
uploaded = files.upload()

Saving best_model_final.h5 to best_model_final.h5
Saving ENG_M (1).wav to ENG_M (1).wav


In [1]:
import h5py
f = h5py.File('best_model_final.h5', 'r')
for key in f['optimizer_weights'].values():
   print(key)
print(f)
f.close()

<HDF5 group "/optimizer_weights/Adam" (7 members)>
<HDF5 file "best_model_final.h5" (mode r)>


In [2]:
data

NameError: name 'data' is not defined

In [3]:
"""Develop an api"""

import librosa
import spacy
import re
from pydub import AudioSegment
from pydub.silence import split_on_silence
import warnings
import en_core_web_sm
from spacy.util import load_model
warnings.filterwarnings('ignore')
import pandas as pd
import speech_recognition as sr
import os
from flask import Flask, request, jsonify, render_template
import numpy as np
import json
from googletrans import Translator
from werkzeug.wrappers import Request, Response

present_tenses = ['VBP', 'VBZ','VBG']
nlp = en_core_web_sm.load()

def extract_spectrum(file_name, file_length=5, bin_factor=4, **kwargs):
    """
    file_length is the target time length of the file in seconds
    bin_factor is the amount by which we enlarge the time bins from 512 samples:
    Extract mel spectrogram from audio file `file_name`
    """

    X, sample_rate = librosa.core.load(file_name)
    file_name = file_name.split('/')[-1]
    target_name = file_name[:-4]
    target_length = int(file_length * sample_rate)
    target = np.zeros(target_length)
    if X.shape[0] >= target_length:
        target = X[:target_length]
    else:
        target[:X.shape[0]] = X
    X = target
    mel = librosa.feature.melspectrogram(X, sr=sample_rate, hop_length=512 * bin_factor).T
    print(file_name, ' loaded with shape ', mel.shape)
    return mel


def open_model(filename="best_model_final.h5"):
    """ Given filename, returns the model
    param filename: filepath for model
    return : pretrained model """

    return load_model(filename)

def predict_from_model(X):
    """ Given X, returns a prediction
    param X: the feature values to consider
    returns pred_pickle: the prediction """

    model = open_model()
    X = extract_spectrum(X).reshape(-1,54,128,1)

    pred = model.predict(X)
    if pred[0][0] <= 0.5:
        return 'female'
    else:
        return 'male'

def get_large_audio_transcription(X):
    """
    Splitting the large audio file into chunks
    and apply speech recognition on each of these chunks
    """
    r = sr.Recognizer()
    # open the audio file using pydub
    sound = AudioSegment.from_wav(X)
    # split audio sound where silence is 700 miliseconds or more and get chunks
    chunks = split_on_silence(sound,
        # experiment with this value for your target audio file
        min_silence_len = 500,
        # adjust this per requirement
        silence_thresh = sound.dBFS-14,
        # keep the silence for 1 second, adjustable as well
        keep_silence=500,
    )
    folder_name = "audio-chunks"
    # create a directory to store the audio chunks
    if not os.path.isdir(folder_name):
        os.mkdir(folder_name)
    whole_text = ""
    # process each chunk
    for i, audio_chunk in enumerate(chunks, start=1):
        # export audio chunk and save it in
        # the `folder_name` directory.
        chunk_filename = os.path.join(folder_name, f"chunk{i}.wav")
        audio_chunk.export(chunk_filename, format="wav")
        # recognize the chunk
        with sr.AudioFile(chunk_filename) as source:
            audio_listened = r.record(source)
            # try converting it to text
            try:
                text = r.recognize_google(audio_listened)
            except sr.UnknownValueError as e:
                print("Error:", str(e))
            else:
                text = f"{text.capitalize()}. "
                whole_text += text
    # return the text for all chunks detected
    return whole_text



def determine_chunks(pronouns, vals, text,present_tenses = present_tenses):
    nlp = spacy.load('en')

    present_tenses = ['VBP', 'VBZ','VBG']

    chunks = []

    iterations = len(pronouns)

    for i in range(iterations):

        start = vals[i]
        stop = None

        if i != (iterations - 1):
            stop = vals[i+1]

        if pronouns[i] != 'I':
            chunks.append((str(text[start:stop]),0))
            continue


        present_verbs_change = [word for word in text[start:stop]
                                if word.tag_ in
                                present_tenses]

        if len(present_verbs_change) == 0:
            chunks.append((str(text[start:stop]),0))
            continue

        chunks.append((str(text[start:stop]),1))

    return chunks

def determine_tense_input(sentence, present_tenses = present_tenses):

    """ Seperates words into chunks based on subjects of the sentence
    and sends them to the determine_chunks function to determine
    if that chunk requires further manipulation or not
    param ....
    .....
    ....
    returns: list of tuples (portion, int) where 1 means the chunk
    requires further manipulation and 0 means it can be left as is """

    text = nlp(sentence)

    present = len([word for word in text
                    if word.tag_ in present_tenses])

    if present == 0 :
        return [(sentence,0)]

    sub_toks = list(sorted([ (i, tok) for i, tok
                            in enumerate(text)
                            if tok.dep_ == "nsubj" ]))


    vals = [x[0] for x in sub_toks]
    pronouns = [str(x[1]) for x in sub_toks]

    if 'I' not in pronouns:
        return [(sentence,0)]

    return determine_chunks(pronouns, vals, text)

def get_translation(output, gender):
    heb_I =  'אני'
    heb_she = 'היא'
    heb_he = 'הוא'
    ind = []
    translate = ''
    lenght = 0
    translator = Translator()
    replacer = 'I'


    for sentence, val in output:
        res = translator.translate(sentence, dest='he').text
        res_list = re.findall(r"[\w]+|['.,!?;]", res)
        if val>0:
            if gender == 'female':
                replacer = 'she'
                heb_replacer = heb_she
            elif gender == 'male':
                replacer = 'he'
                heb_replacer = heb_he
            index = [i for i,x in enumerate(res_list) if x==heb_I][0]
            ind.append(index+lenght)
        lenght += len(res_list)
        sentence_list = re.findall(r"[\w]+|['.,!?;]", sentence)
        sentence = ' '.join([replacer if word == 'I' else word for word in sentence_list])
        translate += " " + sentence
    gender_res = translator.translate(translate, dest='he').text
    gender_res_list = re.findall(r"[\w]+|['.,!?;]", gender_res)
    for i in ind:
        if gender_res_list[i] == heb_replacer:
            gender_res_list[i] = heb_I
        elif gender_res_list[i-1] ==  heb_replacer:
            gender_res_list[i-1] = heb_I
        elif gender_res_list[i+1] ==  heb_replacer:
            gender_res_list[i+1] = heb_I

    return ' '.join(gender_res_list)


app = Flask(__name__)
run_with_ngrok(app) 

#@app.route('/')
#def upload_file():
#    return render_template('C:\\Users\\Ilan avraham\\PycharmProjects\\ProjectAPI\\template.html')

@app.route("/")
def prediction():

    pred = predict_from_model('ENG_M (1).wav')
    chunks = get_large_audio_transcription('ENG_M (1).wav')
    split_sentence = determine_tense_input(chunks)
    output = get_translation(split_sentence, pred)
    return pred, output
 
    

app.run()

ModuleNotFoundError: No module named 'pydub'

In [9]:
!pip install flask-ngrok

In [11]:
from flask_ngrok import run_with_ngrok
from flask import Flask
app = Flask(__name__)
run_with_ngrok(app)   #starts ngrok when the app is run
@app.route("/")
def home():
    return "<h1>Running Flask on Google Colab!</h1>"
  
app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://d824bcf38006.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040
